In [1]:
# Para usar o sleep:
import time
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import cchardet
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
# Contador
from itertools import count

Inicializando o logger:

In [2]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

### Obtendo os links das cervejas brasileiras do _Untappd_:

Fazendo uma requisição GET à página web que contém as cervejas brasileiras na seção _top rated_ da rede social _Untappd_ e transformando o resultado em um objeto BeautifulSoup:

In [19]:
base_url = 'https://untappd.com'
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0"
}
payload = {'country': 'brazil'}

page = requests.get(base_url + '/beer/top_rated', params=payload, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

#print(soup.prettify())

Minerando os links de todas as cervejas brasileiras e armazenando em uma lista:

In [4]:
logger.info("Coletando links de cervejas brasileiras do 'top rated'...")
brazilian_beer_links = [a_tag.get('href') for a_tag in soup.select('div.beer-item > a:first-child')]
logger.info("Feito.")
brazilian_beer_links

2023-09-21 21:30:07,309 - INFO - Coletando links de cervejas brasileiras do 'top rated'...
2023-09-21 21:30:07,324 - INFO - Feito.


['/b/spartacus-brewing-iii-three/4283069',
 '/b/spartacus-brewing-iv-four/4756418',
 '/b/spartacus-brewing-v-five/5285289',
 '/b/hocus-pocus-overdrive/1682105',
 '/b/spartacus-brewing-arete-tomo-2/4542383',
 '/b/salvador-brewing-co-cookie-de-chocolate/2908960',
 '/b/hocus-pocus-elephants-graveyard/2769185',
 '/b/cervejaria-dogma-rizoma/1536694',
 '/b/spartacus-brewing-arete-tomo-1/4098457',
 '/b/hocus-pocus-supersymmetry/1930404',
 '/b/spartacus-brewing-etera-tomo-5/4769967',
 '/b/koala-san-brew-cosmic-blur/3452440',
 '/b/trilha-cervejaria-pao-de-mel/3252352',
 '/b/hocus-pocus-rabbit-hole/2321900',
 '/b/spartacus-brewing-fwiw/3905760',
 '/b/cervejaria-dogma-mosaic-lover/1698497',
 '/b/salvador-brewing-co-engesa-oil-coffee-edition/3036359',
 '/b/spartacus-brewing-barrel-aged-icon-double-chocolate-peanut-butter/5076259',
 '/b/5-elementos-cervejaria-artesanal-coffee-and-pancake-brunch-stout/2546467',
 '/b/mad-dwarf-brewery-md-oloroso-2017/2957221',
 '/b/cervejaria-dogma-morning-gringo/265

In [5]:
number_of_brazilian_beer_links = len(brazilian_beer_links)
logger.info(f"Número total de cervejas no 'top rated' do Brasil: {number_of_brazilian_beer_links}")

2023-09-21 21:30:07,350 - INFO - Número total de cervejas no 'top rated' do Brasil: 50


### Acessando as páginas completas das cervejas com Selenium:

Carregando as credenciais de acesso:

In [6]:
dotenv.load_dotenv()
EMAIL = os.getenv('EMAIL')
PASSWORD = os.getenv('PASSWORD')

Definindo as opções do navegador que o Selenium usará:

In [7]:
options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

Acessando a página completa de cada cerveja do _top rated_ do Brasil clicando no link 'Show more' e minerando os IDs de usuários:

In [8]:
browser = webdriver.Chrome(options=options)

browser.get(base_url + '/login')
browser.find_element(By.NAME,"username").send_keys(EMAIL)
browser.find_element(By.NAME,"password").send_keys(PASSWORD)
time.sleep(30)

all_users = set()
logger.info("Iniciando uma sessão...")
with requests.Session() as session:
    logger.info("A iteração começará agora...")
    iteration_time = Clock("Coleta dos IDs de usuários")
    with tqdm(total=number_of_brazilian_beer_links) as progress_bar:
        progress_bar.set_description("Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil")
        wait = WebDriverWait(browser, 10)
        page_counter = count(1)
        for beer_url in brazilian_beer_links:
            browser.get(base_url + beer_url)
            counter = count(1)
            while True:
                try:
                    element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-href=':feed/showmore']")))
                    browser.execute_script("arguments[0].click();", element)
                    time.sleep(5)
                    logger.info(f"Link 'Show more' da {str(page_counter)[6:-1]}ª página clicado pela {next(counter)}ª vez")
                except:
                    logger.info('Fim da página alcançado!')
                    next(page_counter)
                    break
            full_page = browser.page_source
            a_tag = SoupStrainer('a')
            soup = BeautifulSoup(full_page, 'html.parser', parse_only=a_tag)
            users = set('/user/' + link.get('href').split('/user/')[1].split('/')[0] for link in soup.select("a[href^='/user/']"))
            all_users.update(users)
            logger.info(f"Foram coletados {len(users)} id's de usuários da página completa.")
            progress_bar.update(1)
    iteration_time.stop_watch()
    logger.info("Coleta concluída!")

2023-09-21 21:30:42,028 - INFO - Iniciando uma sessão...
2023-09-21 21:30:42,029 - INFO - A iteração começará agora...
Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:   0%| | 0/50 [00:00<?, 2023-09-21 21:30:51,885 - INFO - Link 'Show more' da 1ª página clicado pela 1ª vez
2023-09-21 21:30:56,941 - INFO - Link 'Show more' da 1ª página clicado pela 2ª vez
2023-09-21 21:31:01,999 - INFO - Link 'Show more' da 1ª página clicado pela 3ª vez
2023-09-21 21:31:07,103 - INFO - Link 'Show more' da 1ª página clicado pela 4ª vez
2023-09-21 21:31:12,188 - INFO - Link 'Show more' da 1ª página clicado pela 5ª vez
2023-09-21 21:31:17,255 - INFO - Link 'Show more' da 1ª página clicado pela 6ª vez
2023-09-21 21:31:22,329 - INFO - Link 'Show more' da 1ª página clicado pela 7ª vez
2023-09-21 21:31:27,417 - INFO - Link 'Show more' da 1ª página clicado pela 8ª vez
2023-09-21 21:31:32,521 - INFO - Link 'Show more' da 1ª página clicado pela 9ª vez
2023-09-21 21:3

2023-09-21 21:38:43,665 - INFO - Link 'Show more' da 7ª página clicado pela 8ª vez
2023-09-21 21:38:48,750 - INFO - Link 'Show more' da 7ª página clicado pela 9ª vez
2023-09-21 21:38:53,852 - INFO - Link 'Show more' da 7ª página clicado pela 10ª vez
2023-09-21 21:38:58,962 - INFO - Link 'Show more' da 7ª página clicado pela 11ª vez
2023-09-21 21:39:04,080 - INFO - Link 'Show more' da 7ª página clicado pela 12ª vez
2023-09-21 21:39:14,507 - INFO - Fim da página alcançado!
2023-09-21 21:39:15,178 - INFO - Foram coletados 542 id's de usuários da página completa.
Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:  14%|▏| 7/50 [08:33<52:2023-09-21 21:39:22,562 - INFO - Link 'Show more' da 8ª página clicado pela 1ª vez
2023-09-21 21:39:27,611 - INFO - Link 'Show more' da 8ª página clicado pela 2ª vez
2023-09-21 21:39:32,666 - INFO - Link 'Show more' da 8ª página clicado pela 3ª vez
2023-09-21 21:39:37,714 - INFO - Link 'Show more' da 8ª página cli

2023-09-21 21:46:46,877 - INFO - Link 'Show more' da 14ª página clicado pela 5ª vez
2023-09-21 21:46:51,932 - INFO - Link 'Show more' da 14ª página clicado pela 6ª vez
2023-09-21 21:46:56,988 - INFO - Link 'Show more' da 14ª página clicado pela 7ª vez
2023-09-21 21:47:02,039 - INFO - Link 'Show more' da 14ª página clicado pela 8ª vez
2023-09-21 21:47:07,106 - INFO - Link 'Show more' da 14ª página clicado pela 9ª vez
2023-09-21 21:47:12,160 - INFO - Link 'Show more' da 14ª página clicado pela 10ª vez
2023-09-21 21:47:17,214 - INFO - Link 'Show more' da 14ª página clicado pela 11ª vez
2023-09-21 21:47:22,259 - INFO - Link 'Show more' da 14ª página clicado pela 12ª vez
2023-09-21 21:47:32,668 - INFO - Fim da página alcançado!
2023-09-21 21:47:33,593 - INFO - Foram coletados 543 id's de usuários da página completa.
Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:  28%|▎| 14/50 [16:51<442023-09-21 21:47:43,095 - INFO - Link 'Show more' da 15ª p

2023-09-21 21:54:59,328 - INFO - Link 'Show more' da 21ª página clicado pela 4ª vez
2023-09-21 21:55:04,383 - INFO - Link 'Show more' da 21ª página clicado pela 5ª vez
2023-09-21 21:55:09,420 - INFO - Link 'Show more' da 21ª página clicado pela 6ª vez
2023-09-21 21:55:14,468 - INFO - Link 'Show more' da 21ª página clicado pela 7ª vez
2023-09-21 21:55:19,501 - INFO - Link 'Show more' da 21ª página clicado pela 8ª vez
2023-09-21 21:55:24,555 - INFO - Link 'Show more' da 21ª página clicado pela 9ª vez
2023-09-21 21:55:29,646 - INFO - Link 'Show more' da 21ª página clicado pela 10ª vez
2023-09-21 21:55:34,698 - INFO - Link 'Show more' da 21ª página clicado pela 11ª vez
2023-09-21 21:55:39,793 - INFO - Link 'Show more' da 21ª página clicado pela 12ª vez
2023-09-21 21:55:49,800 - INFO - Fim da página alcançado!
2023-09-21 21:55:50,597 - INFO - Foram coletados 580 id's de usuários da página completa.
Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasi

Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:  54%|▌| 27/50 [32:23<272023-09-21 22:03:13,478 - INFO - Link 'Show more' da 28ª página clicado pela 1ª vez
2023-09-21 22:03:18,535 - INFO - Link 'Show more' da 28ª página clicado pela 2ª vez
2023-09-21 22:03:23,579 - INFO - Link 'Show more' da 28ª página clicado pela 3ª vez
2023-09-21 22:03:28,630 - INFO - Link 'Show more' da 28ª página clicado pela 4ª vez
2023-09-21 22:03:33,677 - INFO - Link 'Show more' da 28ª página clicado pela 5ª vez
2023-09-21 22:03:38,738 - INFO - Link 'Show more' da 28ª página clicado pela 6ª vez
2023-09-21 22:03:43,799 - INFO - Link 'Show more' da 28ª página clicado pela 7ª vez
2023-09-21 22:03:48,852 - INFO - Link 'Show more' da 28ª página clicado pela 8ª vez
2023-09-21 22:03:53,910 - INFO - Link 'Show more' da 28ª página clicado pela 9ª vez
2023-09-21 22:03:58,976 - INFO - Link 'Show more' da 28ª página clicado pela 10ª vez
2023-09-21 22:04:04,038 - INFO - Link 'S

2023-09-21 22:11:12,541 - INFO - Link 'Show more' da 34ª página clicado pela 6ª vez
2023-09-21 22:11:17,592 - INFO - Link 'Show more' da 34ª página clicado pela 7ª vez
2023-09-21 22:11:22,656 - INFO - Link 'Show more' da 34ª página clicado pela 8ª vez
2023-09-21 22:11:27,713 - INFO - Link 'Show more' da 34ª página clicado pela 9ª vez
2023-09-21 22:11:32,775 - INFO - Link 'Show more' da 34ª página clicado pela 10ª vez
2023-09-21 22:11:37,850 - INFO - Link 'Show more' da 34ª página clicado pela 11ª vez
2023-09-21 22:11:42,898 - INFO - Link 'Show more' da 34ª página clicado pela 12ª vez
2023-09-21 22:11:53,328 - INFO - Fim da página alcançado!
2023-09-21 22:11:54,051 - INFO - Foram coletados 515 id's de usuários da página completa.
Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:  68%|▋| 34/50 [41:12<202023-09-21 22:12:02,596 - INFO - Link 'Show more' da 35ª página clicado pela 1ª vez
2023-09-21 22:12:07,649 - INFO - Link 'Show more' da 35ª p

Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:  80%|▊| 40/50 [48:34<122023-09-21 22:19:24,995 - INFO - Link 'Show more' da 41ª página clicado pela 1ª vez
2023-09-21 22:19:30,041 - INFO - Link 'Show more' da 41ª página clicado pela 2ª vez
2023-09-21 22:19:35,090 - INFO - Link 'Show more' da 41ª página clicado pela 3ª vez
2023-09-21 22:19:40,145 - INFO - Link 'Show more' da 41ª página clicado pela 4ª vez
2023-09-21 22:19:45,198 - INFO - Link 'Show more' da 41ª página clicado pela 5ª vez
2023-09-21 22:19:50,245 - INFO - Link 'Show more' da 41ª página clicado pela 6ª vez
2023-09-21 22:19:55,295 - INFO - Link 'Show more' da 41ª página clicado pela 7ª vez
2023-09-21 22:20:00,351 - INFO - Link 'Show more' da 41ª página clicado pela 8ª vez
2023-09-21 22:20:05,405 - INFO - Link 'Show more' da 41ª página clicado pela 9ª vez
2023-09-21 22:20:10,465 - INFO - Link 'Show more' da 41ª página clicado pela 10ª vez
2023-09-21 22:20:15,542 - INFO - Link 'S

2023-09-21 22:27:22,962 - INFO - Link 'Show more' da 47ª página clicado pela 9ª vez
2023-09-21 22:27:28,030 - INFO - Link 'Show more' da 47ª página clicado pela 10ª vez
2023-09-21 22:27:33,099 - INFO - Link 'Show more' da 47ª página clicado pela 11ª vez
2023-09-21 22:27:43,509 - INFO - Fim da página alcançado!
2023-09-21 22:27:44,269 - INFO - Foram coletados 754 id's de usuários da página completa.
Coletando os IDs de usuários que interagiram com as top 50 cervejas melhor avaliadas do Brasil:  94%|▉| 47/50 [57:02<032023-09-21 22:27:54,266 - INFO - Link 'Show more' da 48ª página clicado pela 1ª vez
2023-09-21 22:27:59,315 - INFO - Link 'Show more' da 48ª página clicado pela 2ª vez
2023-09-21 22:28:04,377 - INFO - Link 'Show more' da 48ª página clicado pela 3ª vez
2023-09-21 22:28:09,428 - INFO - Link 'Show more' da 48ª página clicado pela 4ª vez
2023-09-21 22:28:14,481 - INFO - Link 'Show more' da 48ª página clicado pela 5ª vez
2023-09-21 22:28:19,534 - INFO - Link 'Show more' da 48ª pá

In [15]:
logger.info(f'No total foram coletados {len(all_users)} IDs de usuários!')

users_top_rated_json = {'users_top_rated_brazilian_beer': sorted(all_users)}
path = '../data/'
file_name = "top_rated_brazilian_beer_user_links.json"

logger.info('Salvando a lista de IDs de usuários coletados num arquivo JSON...')
with open(path + file_name, "w", encoding="utf-8") as outfile:
    json.dump(users_top_rated_json, outfile, indent = 4, ensure_ascii=False)
logger.info("Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'top_rated_brazilian_beer_user_links.json'.")

2023-09-21 23:44:38,799 - INFO - No total foram coletados 13458 IDs de usuários!
2023-09-21 23:44:38,804 - INFO - Salvando a lista de IDs de usuários coletados num arquivo JSON...
2023-09-21 23:44:38,814 - INFO - Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'top_rated_brazilian_beer_user_links.json'.


### Filtrando usuários possivelmente brasileiros:

Acessando os dados minerados armazenados em JSON:

In [5]:
with open(path + file_name, "r") as readfile:
    data = json.load(readfile)

Obtendo a lista de todas as cidades do Brasil:

In [9]:
all_users = data['users_top_rated_brazilian_beer']


url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
response = requests.get(url)
brazilian_cities = []

if response.status_code == 200:
    data = response.json()
    for city in data:
        brazilian_cities.append(city["nome"])
else:
    print("Falha na solicitação à API")

Filtrando os IDs de usuários que são de cidades brasileiras ou têm grandes chances de ser (similaridade de 80% pelo menos entre os nomes das localizações e alguma cidade do Brasil) e salvando numa lista filtrada, bem como um dicionário para revisão:

In [62]:
from thefuzz import fuzz

brazilian_users = []
to_review = {}
logger.info("Iniciando uma sessão...")
with requests.Session() as session:
    logger.info("A iteração começará agora...")
    iteration_time = Clock("Filtragem dos IDs de usuários")
    with tqdm(total=len(all_users)) as progress_bar:
        progress_bar.set_description("Filtrando apenas os IDs de usuários que são de cidades brasileiras")
        for user in all_users:
            try:
                page = requests.get(base_url + user, headers=headers)
                soup = BeautifulSoup(page.text, 'html.parser')
                location = soup.select('div.user-details > p.location')[0].get_text()
                if location != '':
                    if location in brazilian_cities:
                        logger.info(f"Usuário brasileiro encontrado ({user}, cidade: {location})! Adicionando à nova lista...")
                        brazilian_users.append(user)
                    else:
                        similarity_threshold = 80
                        for brazilian_city in brazilian_cities:
                            similarity = fuzz.ratio(location, brazilian_city)
                            if similarity >= similarity_threshold:
                                logger.info(f"Possível usuário brasileiro encontrado ({user}, cidade: {location}, {similarity}% de similaridade com {brazilian_city})! Adicionando à nova lista...")
                                brazilian_users.append(user)
                                logger.info("Adicionando também ao dicionário de revisão manual...")
                                to_review[user]= location
                                break
                progress_bar.update(1)
            except IndexError:
                print(f"Elemento não encontrado para o usuário: {user}")
                progress_bar.update(1)
                continue
    iteration_time.stop_watch()
logger.info(f"Operação concluída! A lista filtrada possui {len(brazilian_users)} IDs de usuários.")

2023-09-22 03:35:57,194 - INFO - Iniciando uma sessão...
2023-09-22 03:35:57,195 - INFO - A iteração começará agora...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   0%|         | 36/13458 [00:26<2:19:04,  1.61it/s]2023-09-22 03:36:24,670 - INFO - Possível usuário brasileiro encontrado (/user/1berto_goulart, cidade: Rio Bonito - RJ, 80% de similaridade com Rio Bonito)! Adicionando à nova lista...
2023-09-22 03:36:24,670 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   0%|         | 53/13458 [00:37<2:25:25,  1.54it/s]2023-09-22 03:36:35,272 - INFO - Usuário brasileiro encontrado (/user/31mike, cidade: Colorado)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   1%|         | 97/13458 [01:07<3:00:50,  1.23it/s]2023-09-22 03:37:05,532 - INFO - Possível usuário brasileiro encontrado (/user/ABeerLikeYou, cidade: Paris, 83% de similaridade

Filtrando apenas os IDs de usuários que são de cidades brasileiras:   2%|▏       | 305/13458 [03:43<2:41:16,  1.36it/s]2023-09-22 03:39:41,622 - INFO - Usuário brasileiro encontrado (/user/Am3lia, cidade: São Paulo)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   3%|▏       | 337/13458 [04:08<2:56:38,  1.24it/s]2023-09-22 03:40:06,158 - INFO - Usuário brasileiro encontrado (/user/AndersonBiro, cidade: Franco da Rocha)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   3%|▏       | 339/13458 [04:09<2:45:02,  1.32it/s]2023-09-22 03:40:07,418 - INFO - Usuário brasileiro encontrado (/user/Anderson_Zanin, cidade: São Paulo)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   3%|▏       | 349/13458 [04:16<2:26:11,  1.49it/s]2023-09-22 03:40:14,018 - INFO - Possível usuário brasileiro encontrado (/user/AndreMartins, cidade: Brasil, 80% de similarida

Filtrando apenas os IDs de usuários que são de cidades brasileiras:   6%|▍       | 770/13458 [09:40<2:46:22,  1.27it/s]2023-09-22 03:45:38,649 - INFO - Possível usuário brasileiro encontrado (/user/BernardoMota, cidade: Vila Nova de Gaia, 80% de similaridade com Vila Nova do Piauí)! Adicionando à nova lista...
2023-09-22 03:45:38,649 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   6%|▍       | 788/13458 [09:54<2:12:04,  1.60it/s]2023-09-22 03:45:52,662 - INFO - Possível usuário brasileiro encontrado (/user/BethHabib, cidade: Laranjeiras - RJ, 81% de similaridade com Laranjeiras)! Adicionando à nova lista...
2023-09-22 03:45:52,671 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   6%|▍       | 790/13458 [09:56<2:32:22,  1.39it/s]2023-09-22 03:45:54,138 - INFO - Possível usuário brasileiro encontrado (/user/Betinhojr, cida

Filtrando apenas os IDs de usuários que são de cidades brasileiras:   9%|▌      | 1172/13458 [14:53<2:23:33,  1.43it/s]2023-09-22 03:50:51,787 - INFO - Usuário brasileiro encontrado (/user/CamilaCamui, cidade: Sorocaba)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   9%|▌      | 1175/13458 [14:56<2:27:45,  1.39it/s]2023-09-22 03:50:53,985 - INFO - Possível usuário brasileiro encontrado (/user/Camillo_BN, cidade: Brasil, 80% de similaridade com Brasiléia)! Adicionando à nova lista...
2023-09-22 03:50:53,985 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   9%|▌      | 1178/13458 [14:58<2:24:37,  1.42it/s]2023-09-22 03:50:56,045 - INFO - Usuário brasileiro encontrado (/user/Campelo, cidade: São Paulo)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:   9%|▌      | 1185/13458 [15:03<2:15:14,  1.51it/s]2023-09-2

Elemento não encontrado para o usuário: /user/CervejariaEverBrew


...

2023-09-22 06:19:41,850 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  91%|█████▍| 12224/13458 [2:43:51<17:58,  1.14it/s]2023-09-22 06:19:49,227 - INFO - Usuário brasileiro encontrado (/user/richmtp, cidade: São Paulo)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  91%|█████▍| 12241/13458 [2:44:04<17:01,  1.19it/s]2023-09-22 06:20:02,772 - INFO - Possível usuário brasileiro encontrado (/user/rimenes, cidade: Joao Pessoa, 91% de similaridade com João Pessoa)! Adicionando à nova lista...
2023-09-22 06:20:02,788 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  91%|█████▍| 12247/13458 [2:44:10<21:11,  1.05s/it]2023-09-22 06:20:08,892 - INFO - Possível usuário brasileiro encontrado (/user/rjardim, cidade: Blumenau/SC, 84% de similaridade com Blumenau)! Adicionando à nova list

2023-09-22 06:21:29,538 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  92%|█████▌| 12352/13458 [2:45:35<13:23,  1.38it/s]2023-09-22 06:21:33,818 - INFO - Usuário brasileiro encontrado (/user/rogjorn, cidade: Rio de Janeiro)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  92%|█████▌| 12365/13458 [2:45:46<14:59,  1.21it/s]2023-09-22 06:21:45,009 - INFO - Usuário brasileiro encontrado (/user/ronacarado, cidade: São Paulo)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  92%|█████▌| 12380/13458 [2:45:59<15:03,  1.19it/s]2023-09-22 06:21:57,217 - INFO - Usuário brasileiro encontrado (/user/roruano, cidade: Marília)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  92%|█████▌| 12398/13458 [2:46:13<13:38,  1.30it/s]2023-09-22 06:22:11,906 - INFO - Usuário brasileir

2023-09-22 06:27:21,142 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  95%|█████▋| 12785/13458 [2:51:28<09:21,  1.20it/s]2023-09-22 06:27:26,943 - INFO - Usuário brasileiro encontrado (/user/taisacarvalho, cidade: Rio de Janeiro)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  95%|█████▋| 12792/13458 [2:51:34<09:22,  1.18it/s]2023-09-22 06:27:32,631 - INFO - Possível usuário brasileiro encontrado (/user/tanoeiro, cidade: Brasil, 80% de similaridade com Brasiléia)! Adicionando à nova lista...
2023-09-22 06:27:32,631 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  95%|█████▋| 12800/13458 [2:51:41<09:11,  1.19it/s]2023-09-22 06:27:39,311 - INFO - Possível usuário brasileiro encontrado (/user/tatileitenski, cidade: Curitiba , 82% de similaridade com Muritiba)! Adicionando à 

2023-09-22 06:29:41,519 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  97%|█████▊| 13028/13458 [2:54:47<06:15,  1.14it/s]2023-09-22 06:30:45,233 - INFO - Usuário brasileiro encontrado (/user/toscanoigor, cidade: Belo Horizonte)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  97%|█████▊| 13036/13458 [2:54:54<06:47,  1.04it/s]2023-09-22 06:30:52,862 - INFO - Possível usuário brasileiro encontrado (/user/trevisanrossi, cidade: Brasil, 80% de similaridade com Brasiléia)! Adicionando à nova lista...
2023-09-22 06:30:52,862 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  97%|█████▊| 13058/13458 [2:55:13<05:34,  1.20it/s]2023-09-22 06:31:11,935 - INFO - Possível usuário brasileiro encontrado (/user/tulasi_cordeiro_5391, cidade: Floripa, 80% de similaridade com Floriano)! Adicio

2023-09-22 06:35:04,973 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  99%|█████▉| 13356/13458 [2:59:11<01:24,  1.20it/s]2023-09-22 06:35:09,189 - INFO - Possível usuário brasileiro encontrado (/user/wschmidt, cidade: Brasil, 80% de similaridade com Brasiléia)! Adicionando à nova lista...
2023-09-22 06:35:09,189 - INFO - Adicionando também ao dicionário de revisão manual...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  99%|█████▉| 13367/13458 [2:59:19<01:07,  1.35it/s]2023-09-22 06:35:17,747 - INFO - Usuário brasileiro encontrado (/user/xDanillo94, cidade: São Paulo)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários que são de cidades brasileiras:  99%|█████▉| 13385/13458 [2:59:34<01:01,  1.19it/s]2023-09-22 06:35:31,923 - INFO - Usuário brasileiro encontrado (/user/yamarashi, cidade: Rio de Janeiro)! Adicionando à nova lista...
Filtrando apenas os IDs de usuários 

In [69]:
logger.info(f"Dos {len(brazilian_users)} usuários resultantes, {len(to_review)} ({len(to_review)/len(brazilian_users)*100:.2f}%) foram adicionados com base na similaridade e devem ser revisados.")

2023-09-22 12:02:28,694 - INFO - Dos 1238 usuários resultantes, 535 (43.21%) foram adicionados com base na similaridade e devem ser revisados.


Exportando os dados:

In [73]:
possible_br_users_json = {'possible_br_users': brazilian_users}
possible_br_users_file_name = "possible_brazilian_beer_user_links.json"

logger.info('Salvando a lista de IDs de usuários filtrados num arquivo JSON...')
with open(path + possible_br_users_file_name, "w", encoding="utf-8") as outfile:
    json.dump(possible_br_users_json, outfile, indent = 4, ensure_ascii=False)
logger.info("Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'possible_brazilian_beer_user_links.json'.")


file_to_review = "users_to_review.json"
logger.info('Salvando a lista de IDs de usuários que precisam de revisão num arquivo JSON...')
with open(path + file_to_review, "w", encoding="utf-8") as outfile:
    json.dump(to_review, outfile, indent = 4, ensure_ascii=False)
logger.info("Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'users_to_review.json'.")

2023-09-22 12:18:57,140 - INFO - Salvando a lista de IDs de usuários filtrados num arquivo JSON...
2023-09-22 12:18:57,143 - INFO - Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'possible_brazilian_beer_user_links.json'.
2023-09-22 12:18:57,144 - INFO - Salvando a lista de IDs de usuários que precisam de revisão num arquivo JSON...
2023-09-22 12:18:57,176 - INFO - Exportação concluída! O arquivo resultante se encontra dentro da pasta 'data' com o nome 'users_to_review.json'.
